# Read file

In [66]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from numpy import pi, exp, sqrt
from statsmodels.nonparametric.kernel_regression import KernelReg
from statsmodels.regression.rolling import RollingWLS
import talib as ta
import plotly.graph_objects as go

In [4]:
# !pip install pyarrow

In [261]:
df = pd.read_feather(
    "/Users/nicholasle/Desktop/algoTrading/pytrading/frameworks/freqtrade/user_data/data/binance/DOT_USDT-15m.feather"
)

In [262]:
df.head()

,date,open,high,low,close,volume
0,2021-01-01 00:00:00+00:00,9.2620,9.3703,9.1697,9.2428,957296.63
1,2021-01-01 00:15:00+00:00,9.2428,9.2800,9.1410,9.1670,607590.30
2,2021-01-01 00:30:00+00:00,9.1623,9.1987,9.0373,9.1987,574184.68
3,2021-01-01 00:45:00+00:00,9.1987,9.4031,9.1870,9.2609,733787.79
4,2021-01-01 01:00:00+00:00,9.2628,9.4280,9.2600,9.3129,681561.69


In [263]:
df.shape

(103817, 6)

In [206]:
def detect_signal(series):
    diff = series - series.shift(1)
    conditions_min, conditions_max = [], []
    for i in range(1, len(series)):
        if (np.sign(diff.iloc[i]) == 1.0 and np.sign(diff.iloc[i-1]) == -1.0):
            conditions_min.append(True)
        elif not (np.sign(diff.iloc[i]) == 1.0 and np.sign(diff.iloc[i-1]) == -1.0):
            conditions_min.append(False)
        if np.sign(diff.iloc[i]) == -1.0 and np.sign(diff.iloc[i-1]) == 1.0:
            conditions_max.append(True)
        elif not (np.sign(diff.iloc[i]) == -1.0 and np.sign(diff.iloc[i-1]) == 1.0):
            conditions_max.append(False)
    if (conditions_min[-3] and not conditions_min[-2] and not conditions_min[-1]):
        return 1
    elif (conditions_max[-3] and not conditions_max[-2] and not conditions_max[-1]):
        return -1
    else:
        return 0
# def detect_max(series):
#     diff = series - series.shift(1)
#     conditions = []
#     for i in range(1, len(series)):
#         if np.sign(diff.iloc[i]) == -1.0 and np.sign(diff.iloc[i-1]) == 1.0:
#             conditions.append(True)
#     return 1 if (conditions[-3] and not conditions[-2] and not conditions[-1]) else 0

In [253]:
def calculate_signal(series, bandwidth=np.array([5])):
    """
    Calculate the signal for each window.
    
    """
    #  Use Kernel Regression to create a fitted curve
    kernel_regression = KernelReg([series.values], [series.index.to_numpy()], var_type='c', bw=bandwidth)
    regression_result = kernel_regression.fit([series.index])
    
    # Get smoothed close prices
    smoothed_prices_df = pd.Series(data=regression_result[0], index=series.index)
    
    signal = detect_signal(smoothed_prices_df)
    
    return signal

In [179]:
def backtest(df, capital=1000.0, fee=0.001):
    print("Starting backtest...")
    print(f"Starting balance: ${capital:.2f}")
    stake = 0.5 * capital
    in_market: bool = False
    quantity: float = 0.0
    for i in range(1, len(df)):
        if not in_market and df.iloc[i]['enter_long'] == 1:
            quantity += (stake / df.iloc[i-1]['close'])
            capital -= (stake + fee * stake)
            in_market = True
        if in_market and df.iloc[i]['exit_long'] == 1:
            capital += ((quantity * df.iloc[i-1]['close']) - fee * (quantity * df.iloc[i-1]['close'])) 
            quantity = 0.0
            stake = 0.5 * capital
            in_market = False
    
    # Report final value:
    print(f"Closing balance: ${capital:.2f}")
        

In [198]:
def create_plot(df):
    fig = go.Figure()
    #  Close
    fig.add_trace(
        go.Scatter(x=df.index, y=df['close'], line=dict(color="blue", width=1), name="Close")
    )
    #  Kernel Regression
    fig.add_trace(
        go.Scatter(x=df.index, y=df['visual_long'], mode='markers', marker_symbol='triangle-up', marker_color='green', name="Enter")
    )
    fig.add_trace(
        go.Scatter(x=df.index, y=df['visual_exit'], mode='markers', marker_symbol='triangle-down', marker_color='red', name="Exit")
    )
    fig.show()

In [ ]:
def trade(df):
    pass

In [271]:
df1 = df[:50000].copy()

In [272]:
df1['signal'] = df1['close'].rolling(window=10, closed='left').apply(calculate_signal)

In [273]:
mask_enter = (df1['signal'] == 1.0)
mask_exit = (df1['signal'] == -1.0)

In [274]:
df1.loc[mask_enter, 'visual_long'] = df1.loc[mask_enter, 'close']
df1.loc[mask_exit, 'visual_exit'] = df1.loc[mask_exit, 'close']

In [275]:
df1.loc[mask_enter, 'enter_long'] = 1
df1.loc[mask_exit, 'exit_long'] = 1

In [276]:
backtest(df1)

Starting backtest...
Starting balance: $1000.00
Closing balance: $826.63
